# Test OpenAI Agents SDK
- Basic SDK usage
- Use langfuse for prompt repository, evaluation, observability, user feedback
- Run batches of prompt async against lists and dataframes
- Implement a workflow to write a daily AI newsletter

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import langfuse
from langfuse import get_client
from langfuse import Langfuse
from langfuse.openai import openai
# from langfuse.openai import AsyncOpenAI
import logfire
from llm import LangfuseClient

from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent  # methods to apply prompts async to large batches
from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent



In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")
print(f"LangFuse           {langfuse.version.__version__}")
print(f"Logfire            {logfire.__version__}")


OpenAI:            1.107.0
OpenAI Agents SDK  0.2.11
Pydantic           2.11.7
LangFuse           3.3.4
Logfire            4.7.0


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

16:32:32 | NewsletterAgent.newsletter_agent | INFO | Test info message
16:32:32 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
16:32:32 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# Configure logfire instrumentation.
# OpenAI used logfire from Pydantic AI and we need this for langfuse to handle some traces sent automatically by Agents SDK
logfire.configure(
    service_name="my_agent_service", 
    send_to_logfire=False,
    console=False,  # Disable console output
)
logging.getLogger("logfire").setLevel(logging.WARNING)
# Set logfire logger to WARNING level to reduce output
logfire.instrument_openai_agents()

# Langfuse Prompt Repo

In [6]:
# initialize langfuse for observability
# git clone https://github.com/langfuse/langfuse.git
# cd langfuse
# go to localhost:3000
# set up an org, project, get API keys and put in .env

lf_client = get_client()
 
# Verify connection
if lf_client.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")# Get production prompts
prompt = lf_client.get_prompt("newsagent/headline_classifier")

# Get prompt from repository by label
# You can use as many labels as you'd like to identify different deployment targets
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="production")
print(prompt.prompt, "\n")
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="latest")
print(prompt.prompt, "\n")

# Get by version number, usually not recommended as it requires code changes to deploy new prompt versions
prompt = lf_client.get_prompt("newsagent/headline_classifier", version=1)
print(prompt.prompt, "\n")


Langfuse client is authenticated and ready!
[{'type': 'message', 'role': 'system', 'content': 'You are a content-classification assistant that labels news headlines as AI-related or not.\nReturn JSON that matches the provided schema\n\nA headline is AI-related if it mentions (explicitly or implicitly):\n- Core AI models: machine learning, neural / deep / transformer networks\n- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media\n- AI hardware, GPU chip supply, AI data centers and infrastructure\n- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.\n- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.\n- New AI products and AI integration into existing products/services\n- AI policy / ethics / safety / regulation / analysis\n- Research results related to AI\n- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)\n- AI market and business developments, funding rounds, partnerships centered

In [7]:
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")
print("system prompt")
print(system_prompt)
print() 

print("user prompt")
print(user_prompt)
print() 

print("model")
print(model)

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/headline_classifier' from Langfuse
INFO:llm:Parsed prompt 'newsagent/headline_classifier': model=gpt-5-mini, system_len=1093, user_len=48


system prompt
You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other news with a significant AI component

Not AI-related: business software, cr

# Basic usage
- Run a prompt using agents
- Sessions
- Route through Langfuse for observability
- Save logs
- View traces and evals


In [8]:
# Get current `production` version of the prompt via raw langfuse client
system_prompt = lf_client.get_prompt("swallow/system")
 
# Insert variables into prompt template
# compiled_prompt = prompt.compile(criticlevel="expert", movie="Dune 2")

system_prompt.prompt


'You are an expert on airspeed velocities of swallows. Answer questions about swallow flight speeds with authority and humor when appropriate.'

In [9]:
# delete this local db to store agent sessions 
[os.remove(f) for f in glob.glob('swallow.db*') if os.path.exists(f)]


[None, None, None]

In [10]:
# run a chat using the OpenAI Agent class with a persistent session (and through langfuse for observability)

user_prompt = lf_client.get_prompt("swallow/user1").compile()
print(user_prompt)

openai_client = AsyncOpenAI()

# async def main():
agent = Agent(
    name="Assistant",
    instructions=system_prompt.prompt,
    model=OpenAIResponsesModel(model="gpt-4.1", openai_client=openai_client),
)

# 1) Create (or reuse) a session. Use a durable DB path if you want persistence.
session = SQLiteSession("test_swallow_chat", "swallow.db")

result = await Runner.run(agent, user_prompt, session=session)
display(Markdown(result.final_output))

# loop = asyncio.get_running_loop()
# await loop.create_task(main())


What is the airspeed velocity of an unladen swallow?



Ah, the classic question! "What is the airspeed velocity of an unladen swallow?"

The answer, as any aficionado of Monty Python and curious ornithologist knows, depends on a crucial follow-up: **African or European swallow?**

For the European Swallow (*Hirundo rustica*), the commonly cited figure—thanks to scientific studies and one particularly famous bridge-guarding knight—is approximately **11 meters per second**, or **about 24 miles per hour (38 km/h)** when cruising. 

African swallows (*various species like Cecropis daurica*) can be a bit more, well, flighty, but typically fly at similar speeds. That said, African swallows are actually less commonly studied in this regard, perhaps because they wisely avoid answering riddles on ancient bridges.

So, to summarize:
- **European Swallow (unladen):** ~11 m/s (24 mph)
- **African Swallow (unladen):** Somewhere in the same ballpark, but who’s counting? (Well, I am, but data is less precise.)

And remember: carrying a coconut would definitely slow them down. Please don't try this at home—or over any chasms.

In [11]:
# 3) Next turns — just keep reusing the same session
result = await Runner.run(agent, "explain how that number was measured / computed", session=session)

display(Markdown(result.final_output))

Absolutely—let’s soar into the details!

**How Was the Airspeed Velocity of an Unladen Swallow Measured or Computed?**

### **1. Ornithological Studies**
Bird flight speeds have been measured and analyzed by ornithologists using a variety of methods. For swallows, the most reliable studies involve:

**a) Field Observations with Stopwatches and Rulers**  
Researchers have tracked swallows in flight over known distances (sometimes between two landmarks). By timing how long it took a swallow to fly from Point A to Point B, they calculated average cruising speed.

**b) Cinematography and High-Speed Cameras**  
In more recent times, ornithologists have used high-speed cameras to film birds in flight and precisely measure their movement frame-by-frame over set distances.

**c) Radar Tracking**  
Some studies use radar to track small birds in flight, which is accurate but typically reserved for larger migration studies.

### **2. Theoretical Calculations**
The oft-quoted figure—**11 m/s (24 mph)**—comes from a mix of measurement and calculation, most famously chronicled by Charles Pennycuick, an expert in bird flight dynamics.

He approached the problem like this:
- **Estimate the wing-beat frequency:** For a European Swallow, this is about 15 beats per second.
- **Estimate the distance traveled per wing beat:** Pennycuick estimated around 0.73 meters forward per beat.
- **Do the math:**  
    ```
    Airspeed = Wingbeat frequency × Distance per beat
    Airspeed ≈ 15 Hz × 0.73 m
    Airspeed ≈ 11 m/s
    ```

### **3. Published Scientific Sources**
- **Charles J. Pennycuick (Bird Flight Performance, Oxford 1989)** and similar works provide the math and observational backing for these results.
- Further refinements come from field guides and migration studies, which confirm swallows’ steady cruising speeds fit right around that 11 m/s mark.

---

### **In Short:**
- **Part field measurement, part mathematics, and part ornithologist with a stopwatch and a solid sense of humor.**
- If you ever feel like timing a swallow yourself, remember: It’s best to use modern optics. Or, y’know, just ask an expert (that’s me)!

And now you, too, can cross the Bridge of Death with scientific confidence.

### View langfuse trace in the langfuse console
can see time, cost, potentially run tests to eval performance.

![langfuse_trace.png](langfuse_trace.png)


# More advanced usage
- Structured JSON outputs, enables validation and safe passing downstream over long pipelines
- Map prompts to larger data sets asynchronously (e.g. send parallel batches of 50)


In [12]:
# Pydantic output class for classifying headlines - request returning values in this schema
class ClassificationResult(BaseModel):
    """A single headline classification result"""
    input_str: str = Field(description="The original headline text")
    output: bool = Field(description="Whether the headline is AI-related")

class ClassificationResultList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResult] = Field(description="List of classification results")


In [13]:
prompt_name = 'newsagent/headline_classifier'
lf_prompt = lf_client.get_prompt(prompt_name)
print(lf_prompt.prompt, end="\n")
print()

system_prompt = lf_prompt.prompt[0]['content']
print('system prompt\n', system_prompt, end="\n")
print()

user_prompt = lf_prompt.prompt[1]['content']
print('user prompt\n', user_prompt, end="\n")

config = lf_prompt.config if hasattr(lf_prompt, 'config') else {}
print ('config\n', config, end="\n")

model = config.get("model", 'gpt-5')
print('model\n', model, end="\n")


[{'type': 'message', 'role': 'system', 'content': 'You are a content-classification assistant that labels news headlines as AI-related or not.\nReturn JSON that matches the provided schema\n\nA headline is AI-related if it mentions (explicitly or implicitly):\n- Core AI models: machine learning, neural / deep / transformer networks\n- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media\n- AI hardware, GPU chip supply, AI data centers and infrastructure\n- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.\n- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.\n- New AI products and AI integration into existing products/services\n- AI policy / ethics / safety / regulation / analysis\n- Research results related to AI\n- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)\n- AI market and business developments, funding rounds, partnerships centered on AI\n- Any other news with a significant 

In [14]:
# send single prompts via LLMAgent asking for ClassificationResult structured output
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")

classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResult,
    model=model,
    verbose=True,
    logger=logger
)

test_headlines = [
    "AI Is Replacing Online Moderators, But It's Bad at the Job",
    "Baby Trapped in Refrigerator Eats Own Foot",
    "Machine Learning Breakthrough in Medical Diagnosis",
    "Local Restaurant Opens New Location",
    "ChatGPT Usage Soars in Educational Settings"
]

result = await classifier.run_prompt(input_str=test_headlines[0])
print(result)
result = await classifier.run_prompt(input_str=test_headlines[1])
print(result)



INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/headline_classifier' from Langfuse
INFO:llm:Parsed prompt 'newsagent/headline_classifier': model=gpt-5-mini, system_len=1093, user_len=48
16:11:43 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safet

input_str="AI Is Replacing Online Moderators, But It's Bad at the Job" output=True


16:11:52 | NewsletterAgent.newsletter_agent | INFO | Result: input_str='Baby Trapped in Refrigerator Eats Own Foot' output=False


input_str='Baby Trapped in Refrigerator Eats Own Foot' output=False


In [15]:
# LLMAgent in llm.py has multiple ways to request stuff
# Suppose we have 1000 headlines in a dataframe and we want to apply a prompt to each one.
# Some stuff we might want
# - structured output, like ideally apply prompts to this column and put results in a new column
# - output validation, so llm doesn't e.g. transpose rows or skip rows
# - batching , don't send 1000 at once but don't send a single headline with a large prompt 1000 times
# - concurrency / async processing, send many batches at once (but maybe specify some max concurrency)
# - retry logic with exponential backoff
# LLMagent supports
# - run_prompt_dict to use kwargs
# - prompt_dict to send a dict return an object or list , might also just run_prompt(**mydict)
# - prompt_batch to map prompt to a list and return structured object
# - filter_dataframe, map prompt to a Pandas DataFrame and return a Series for assignment

# note different output type
classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResultList,
    model=model,
    verbose=True,
    logger=logger
)

# Format headlines as a single string for batch processing
headlines_str = str(test_headlines)
result = await classifier.prompt_dict({'input_str': headlines_str})
print(f"Batch result: {result}")


16:11:52 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a content-classification assistant that labels news headlines as AI-related or not.
Return JSON that matches the provided schema

A headline is AI-related if it mentions (explicitly or implicitly):
- Core AI models: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: GPT-5, Gemini, Claude, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, Dario Amodei, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other

Batch result: results_list=[ClassificationResult(input_str="AI Is Replacing Online Moderators, But It's Bad at the Job", output=True), ClassificationResult(input_str='Baby Trapped in Refrigerator Eats Own Foot', output=False), ClassificationResult(input_str='Machine Learning Breakthrough in Medical Diagnosis', output=True), ClassificationResult(input_str='Local Restaurant Opens New Location', output=False), ClassificationResult(input_str='ChatGPT Usage Soars in Educational Settings', output=True)]


In [16]:
# make batches and send multiple in parallel
headlines_df = pd.read_csv("test_headlines.csv")
headlines_df


,id,src,title,url
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...
...,...,...,...,...
245,758,NewsAPI,"Reframing Jensen’s Law: ‘Buy more, make more’ ...",https://siliconangle.com/2025/08/30/reframing-...
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...


In [17]:
# filter_dataframe , assign to a column with multiple calls with async concurrency, retry

FILTER_SYSTEM_PROMPT, FILTER_USER_PROMPT, model = LangfuseClient().get_prompt("newsagent/filter_urls")

# output class for classifying headlines
class ClassificationResultId(BaseModel):
    """A single headline classification result"""
    id: int = Field("The news item id")
    input_str: str = Field(description="The original headline title")
    output: bool = Field(description="Whether the headline title is AI-related")

class ClassificationResultIdList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResultId] = Field(description="List of classification results")


classifier = LLMagent(
    system_prompt=FILTER_SYSTEM_PROMPT,
    user_prompt=FILTER_USER_PROMPT,
    output_type=ClassificationResultIdList,  
    model=model,  # Use a valid model
    verbose=False,
    logger=logger
)

headlines_df['isAI'] = await classifier.filter_dataframe(headlines_df[["id", "title"]])

headlines_df



INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=465, user_len=954


,id,src,title,url,isAI
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...,False
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...,True
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...,True
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...,True
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...,True
...,...,...,...,...,...
245,758,NewsAPI,"Reframing Jensen’s Law: ‘Buy more, make more’ ...",https://siliconangle.com/2025/08/30/reframing-...,True
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...,False
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...,False
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...,False


In [18]:
display(headlines_df.loc[headlines_df['isAI']])
display(headlines_df.loc[~headlines_df['isAI']])


,id,src,title,url,isAI
1,10,Ars Technica,"With new in-house models, Microsoft lays the g...",https://arstechnica.com/ai/2025/08/with-new-in...,True
2,16,Ars Technica,Google improves Gemini AI image editing with “...,https://arstechnica.com/ai/2025/08/google-impr...,True
3,20,Ars Technica,Google warns that mass data theft hitting Sale...,https://arstechnica.com/security/2025/08/googl...,True
4,23,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...,True
5,31,Bloomberg,Opinion: China Just Got a Big Leg Up in the AI...,https://www.bloomberg.com/news/videos/2025-08-...,True
...,...,...,...,...,...
241,747,Washington Post,How we tested AI search toolsDetails on the me...,https://www.washingtonpost.com/technology/2025...,True
242,750,NewsAPI,AI stethoscope could detect major heart condit...,https://www.bbc.com/news/articles/c2l748k0y77o,True
243,751,NewsAPI,Robinhood CEO Says AI Will Make Investing 'Muc...,https://biztoc.com/x/485316c3406a106c,True
244,756,NewsAPI,Trade Vector AI: How Trade Vector Artificial I...,https://www.globenewswire.com/news-release/202...,True


,id,src,title,url,isAI
0,0,Ars Technica,GitHub will be folded into Microsoft proper as...,https://arstechnica.com/gadgets/2025/08/github...,False
7,38,Bloomberg,Dell Falls After Reporting Tighter Profit Marg...,https://www.bloomberg.com/news/articles/2025-0...,False
10,44,Bloomberg,Vercel Triples Valuation to $9 Billion With Ac...,https://www.bloomberg.com/news/articles/2025-0...,False
11,46,Bloomberg,Bain Is Said to Draw Chinese Bidders for $4 Bi...,https://www.bloomberg.com/news/articles/2025-0...,False
14,59,Business Insider,The best college laptops of 2025: Top models f...,https://www.businessinsider.com/guides/tech/be...,False
...,...,...,...,...,...
239,742,Washington Post,Why so few Americans read for pleasureThe decl...,https://www.washingtonpost.com/technology/2025...,False
246,759,NewsAPI,Zeta Global (ZETA) Target Raised by Goldman as...,https://finance.yahoo.com/news/zeta-global-zet...,False
247,763,NewsAPI,Luis Enrique names his squad to face Toulouse,https://onefootball.com/en/news/luis-enrique-n...,False
248,773,NewsAPI,CorelDRAW Graphics Suite 2025 v26.2.0.170,https://post.rlsbb.cc/coreldraw-graphics-suite...,False


In [19]:
# test various models for speed, accuracy, cost
# see costs under tracing
# http://localhost:3000/
# 'gpt-5-mini' 9.5¢' 16.8¢, 'gpt-4.1-mini 3.2¢' 
models = ['gpt-5-mini', 'gpt-5-nano', 'gpt-4.1', 'gpt-4.1-mini',]

# a ground truth to compare with (gpt-5)
correct_df = pd.read_csv("headline_classifier_ground_truth.csv")

result_tuples = []

for m in models:
    print(f"Starting evaluation for {m}...")
    
    # Start timing
    start_time = time.time()
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    
    with lf_client.start_as_current_span(name=f"batch_classification_{m}_{timestamp}") as span:
        classifier = LLMagent(system_prompt,
                              user_prompt,
                              ClassificationResultList,
                              m,
                              verbose=False)
    
        # Run classification with tracing
        classification_result = await classifier.prompt_batch(
            list(headlines_df['title'].to_list())
        )
        
        # Add span metadata
        span.update(
            input={"headlines_count": len(headlines_df)},
            metadata={"model": m}
        )
        
    # Calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # analyze results
    result_df = pd.DataFrame([(z.input_str, z.output) 
                              for z in classification_result], 
                             columns=["input", "output"])
    
    
    # Merge with ground truth to compare results
    comparison_df = result_df.merge(correct_df, on='input', suffixes=('_predicted', '_correct'))
    
    # Calculate accuracy
    comparison_df['is_correct'] = comparison_df['output_predicted'] == comparison_df['output_correct']
    accuracy = comparison_df['is_correct'].mean()
    correct_count = comparison_df['is_correct'].sum()
    total_count = len(comparison_df)
    
    # Find differences
    differences_df = comparison_df[~comparison_df['is_correct']].copy()
    
    print(f"Completed {m} in {elapsed_time:.2f} seconds")
    print(f"Accuracy: {correct_count}/{total_count} = {accuracy:.3f} ({accuracy*100:.1f}%)")

    if len(differences_df) > 0:
        print(f"Found {len(differences_df)} incorrect predictions:")
        print("-" * 80)
        for idx, row in differences_df.iterrows():
            print(f"Input: {row['input']}")
            print(f"Predicted: {row['output_predicted']}")
            print(f"Correct:   {row['output_correct']}")
            print("-" * 40)
    else:
        print("🎉 Perfect accuracy! No incorrect predictions.")
    
    print()  # Empty line for readability
    
    # Create tuple with (model_name, df, elapsed_time, accuracy, differences_df)
    result_tuples.append((m, result_df, elapsed_time, accuracy, differences_df))

# Summary comparison
print("=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
summary_data = []
for model_name, df, elapsed_time, accuracy, differences_df in result_tuples:
    rate = len(df) / elapsed_time if elapsed_time > 0 else 0
    summary_data.append({
        'Model': model_name,
        'Accuracy': f"{accuracy:.3f}",
        'Accuracy %': f"{accuracy*100:.1f}%", 
        'Correct': f"{int(accuracy * len(df))}/{len(df)}",
        'Time (s)': f"{elapsed_time:.2f}",
        'Rate (pred/s)': f"{rate:.1f}",
        'Errors': len(differences_df)
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Find most common errors across all models
print("\n" + "=" * 60)
print("MOST COMMON ERRORS ACROSS ALL MODELS")
print("=" * 60)
all_errors = []
for model_name, _, _, _, differences_df in result_tuples:
    for _, row in differences_df.iterrows():
        all_errors.append({
            'input': row['input'],
            'predicted': row['output_predicted'],
            'correct': row['output_correct'],
            'model': model_name
        })

if all_errors:
    error_df = pd.DataFrame(all_errors)
    error_counts = error_df.groupby('input').size().sort_values(ascending=False)
    
    print("Headlines that multiple models got wrong:")
    for headline, count in error_counts.head(10).items():
        if count > 1:  # Only show errors made by multiple models
            models_wrong = error_df[error_df['input'] == headline]['model'].tolist()
            predicted_values = error_df[error_df['input'] == headline]['predicted'].unique()
            correct_value = error_df[error_df['input'] == headline]['correct'].iloc[0]
            
            print(f"\n❌ Error in {count}/{len(models)} models: {', '.join(models_wrong)}")
            print(f"   Headline: {headline}")
            print(f"   Predicted: {predicted_values}")
            print(f"   Correct: {correct_value}")

lf_client.flush()



INFO:llm:Processing 10 batches with concurrency 16


Starting evaluation for gpt-5-mini...


INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-5-mini in 59.10 seconds
Accuracy: 232/243 = 0.955 (95.5%)
Found 11 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Fine-Tune Your Feed and Get News You Can Use
Predicted: True
Correct:   False
----------------------------------------
Input: Framework unveils a second-generation Framework Laptop 16 with a swappable Nvidia RTX 5070 GPU, an industry first, shipping in November 2025 for $2,199+
Predicted: False
Correct:   True
-------

INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-5-nano in 110.42 seconds
Accuracy: 217/222 = 0.977 (97.7%)
Found 5 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Nvidia CFO Colette Kress says Q2 “net other income” was $2.2B, “driven by gains in a publicly-held equity security”, which refers to Nvidia's CoreWeave position
Predicted: True
Correct:   False
----------------------------------------
Input: Google’s first Gemini smart home speaker detailed in leak
Predicted: True
Cor

INFO:llm:Processing 10 batches with concurrency 16


Completed gpt-4.1 in 27.31 seconds
Accuracy: 239/248 = 0.964 (96.4%)
Found 9 incorrect predictions:
--------------------------------------------------------------------------------
Input: Gen Z are eyeing up 'secure' healthcare jobs to AI-proof their careers, but be warned: chiropractors, doctors and paramedics are the unhappiest workers
Predicted: True
Correct:   False
----------------------------------------
Input: Why do we keep making robots dance?
Predicted: True
Correct:   False
----------------------------------------
Input: Show HN: Hacker News em dash user leaderboard pre-ChatGPT
Predicted: True
Correct:   False
----------------------------------------
Input: Filing: Nvidia reveals its top two customers accounted for 39% of its Q2 revenue, up from 25% in Q2, 2024
Predicted: True
Correct:   False
----------------------------------------
Input: Dutch web design automation startup Framer raised $100M led by Meritech and Atomico with Accel participation at a $2B valuation, after r

In [20]:
# access results:
# 'gpt-5-mini' 9.5¢' gpt-5-nano 4.2¢ , gpt-4.1 16.8¢, 'gpt-4.1-mini 3.2¢' 
# note that you can get faster cheaper results with good accuracy
for model_name, df, elapsed_time, accuracy, error_df in result_tuples:
    print(f"{model_name}: {len(df)} results in {elapsed_time:.2f}s, accuracy {accuracy:.3f}")

gpt-5-mini: 250 results in 59.10s, accuracy 0.955
gpt-5-nano: 250 results in 110.42s, accuracy 0.977
gpt-4.1: 250 results in 27.31s, accuracy 0.964
gpt-4.1-mini: 250 results in 17.19s, accuracy 0.962


# Run Agent Worfklow

# test agent



In [6]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
session_id = 'test_newsletter_20250922162121167438'
# Create agent with persistent state
if 'session_id' in vars():
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, db_path="newsletter_agent.db")
    state.load_from_db("step_04_extract_summaries")
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, db_path="newsletter_agent.db") 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)
    state.serialize_to_db("initialize")

16:35:32 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Using provided state with 0 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250922162121167438
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250922162121167438


In [13]:
state=state.load_from_db("step_04_extract_summaries")



In [8]:
session_id 
# 'test_newsletter_20250922162121167438'

'test_newsletter_20250922162121167438'

In [16]:
dir(state)


['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_setattr_handl

In [22]:
state.step_05_cluster_by_topic



<StepStatus.NOT_STARTED: 'not_started'>

In [14]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


16:38:30 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting check_workflow_status
16:38:30 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Completed check_workflow_status


⏱️  Total execution time: 8.60s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch Urls

Per-step:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Rate Articles — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (fetch URLs) now?


In [10]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


16:21:54 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting check_workflow_status
16:21:54 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Completed check_workflow_status
16:21:59 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting Step 1: Gather URLs
2025-09-22 16:21:59,006 - fetcher_4850573136 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-22 16:21:59,016 - fetcher_4850573136 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-22 16:21:59,016 - fetcher_4850573136 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-22 16:21:59,016 - fetcher_4850573136 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-22 16:21:59,017 - fetcher_4850573136 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-22 16:21:59,017 - fetcher_4850573136 - DEBU

2025-09-22 16:22:01,846 - fetcher_4850573136 - INFO - scrape_url(https://www.businessinsider.com/tech)
2025-09-22 16:22:01,846 - fetcher_4850573136 - INFO - scraping https://www.businessinsider.com/tech to download/sources
2025-09-22 16:22:01,847 - fetcher_4850573136 - INFO - File already exists: download/sources/Business_Insider.html
2025-09-22 16:22:01,847 - fetcher_4850573136 - INFO - [fetch_html] Parsing HTML file: download/sources/Business_Insider.html
2025-09-22 16:22:01,872 - fetcher_4850573136 - INFO - [fetch_html] Parsed HTML file: download/sources/Business_Insider.html
2025-09-22 16:22:01,873 - fetcher_4850573136 - INFO - [fetch_html] HTML fetch successful for Business Insider: 17 articles
2025-09-22 16:22:01,873 - fetcher_4850573136 - INFO - [fetch_html] Fetching HTML from Washington Post: https://www.washingtonpost.com/technology/innovations/
2025-09-22 16:22:01,873 - fetcher_4850573136 - INFO - [fetch_html] Source dict for Washington Post: {'type': 'html', 'url': 'https://

2025-09-22 16:22:02,194 - fetcher_4850573136 - INFO - [fetch_html] Parsed HTML file: download/sources/FT.html
2025-09-22 16:22:02,194 - fetcher_4850573136 - INFO - [fetch_html] HTML fetch successful for FT: 103 articles
2025-09-22 16:22:02,194 - fetcher_4850573136 - INFO - [fetch_html] Source dict for The Verge: {'type': 'html', 'url': 'https://www.theverge.com/ai-artificial-intelligence', 'filename': 'The_Verge', 'include': ['^https://www.theverge.com/news'], 'rss': 'https://www.theverge.com/rss/ai-artificial-intelligence/index.xml'}
2025-09-22 16:22:02,195 - fetcher_4850573136 - INFO - Starting scrape_source https://www.theverge.com/ai-artificial-intelligence, The_Verge
2025-09-22 16:22:02,195 - fetcher_4850573136 - INFO - scrape_url(https://www.theverge.com/ai-artificial-intelligence)
2025-09-22 16:22:02,195 - fetcher_4850573136 - INFO - scraping https://www.theverge.com/ai-artificial-intelligence to download/sources
2025-09-22 16:22:02,195 - fetcher_4850573136 - INFO - File already

,source,url
0,Ars Technica,20
1,Bloomberg,33
2,Business Insider,17
3,FT,103
4,Feedly AI,71
5,Hacker News,30
6,HackerNoon,50
7,New York Times,29
8,NewsAPI,45
9,Reddit,57


✅ Completed Step 1: Gathered 643 URLs from 17 RSS sources


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",0
1,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,1
2,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",2
3,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,3
4,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,4
...,...,...,...,...,...,...
638,NewsAPI,AI runs on power. But power isn’t moving fast ...,https://www.wnd.com/2025/09/ai-runs-power-powe...,2025-09-21T19:19:36Z,NaN,638
639,NewsAPI,The Irish Times view on training and apprentic...,https://www.irishtimes.com/opinion/editorials/...,2025-09-21T19:31:00Z,NaN,639
640,NewsAPI,The Race to Reinvent U.S. Nuclear Power,https://oilprice.com/Alternative-Energy/Nuclea...,2025-09-21T19:00:00Z,NaN,640
641,NewsAPI,Reese Witherspoon Says Hollywood Will ‘Change ...,https://variety.com/2025/tv/news/reese-withers...,2025-09-21T19:15:52Z,NaN,641


16:22:02 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Completed Step 1: Gathered 643 articles
16:22:03 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting check_workflow_status
16:22:03 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Completed check_workflow_status


⏱️  Total execution time: 16.54s
📊 Final result:
Step 1 (Fetch URLs) completed.

Summary:
- Collected 643 articles from 17 sources (RSS only).
- Stored in persistent state.

Updated workflow status:
- Progress: 11.1% (1/9 complete)
- Next step: Step 2 — Filter URLs (to AI-related content)

Would you like me to proceed with Step 2 (filter URLs to AI-related content) now?


In [11]:
state 


NewsletterAgentState(headline_data=[{'source': 'Ars Technica', 'title': 'OpenAI and Nvidia’s $100B AI plan will require power equal to 10 nuclear reactors', 'url': 'https://arstechnica.com/ai/2025/09/openai-and-nvidias-100b-ai-plan-will-require-power-equal-to-10-nuclear-reactors/', 'published': 'Mon, 22 Sep 2025 19:17:28 +0000', 'rss_summary': '"This is a giant project," Nvidia CEO said of new 10-gigawatt AI infrastructure deal.', 'id': 0}, {'source': 'Ars Technica', 'title': 'DeepMind AI safety report explores the perils of “misaligned” AI', 'url': 'https://arstechnica.com/google/2025/09/deepmind-ai-safety-report-explores-the-perils-of-misaligned-ai/', 'published': 'Mon, 22 Sep 2025 18:18:00 +0000', 'rss_summary': 'DeepMind releases version 3.0 of its AI Frontier Safety Framework with new tips to stop bad bots.', 'id': 1}, {'source': 'Ars Technica', 'title': 'Science journalists find ChatGPT is bad at summarizing scientific papers', 'url': 'https://arstechnica.com/ai/2025/09/science-j

In [12]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",0
1,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,1
2,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",2
3,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,3
4,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,4
...,...,...,...,...,...,...
638,NewsAPI,AI runs on power. But power isn’t moving fast ...,https://www.wnd.com/2025/09/ai-runs-power-powe...,2025-09-21T19:19:36Z,NaN,638
639,NewsAPI,The Irish Times view on training and apprentic...,https://www.irishtimes.com/opinion/editorials/...,2025-09-21T19:31:00Z,NaN,639
640,NewsAPI,The Race to Reinvent U.S. Nuclear Power,https://oilprice.com/Alternative-Energy/Nuclea...,2025-09-21T19:00:00Z,NaN,640
641,NewsAPI,Reese Witherspoon Says Hollywood Will ‘Change ...,https://variety.com/2025/tv/news/reese-withers...,2025-09-21T19:15:52Z,NaN,641


In [13]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
3,FT,103
4,Feedly AI,71
9,Reddit,57
12,The Register,50
6,HackerNoon,50
8,NewsAPI,45
1,Bloomberg,33
16,Washington Post,31
5,Hacker News,30
7,New York Times,29


In [14]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


16:22:07 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting Step 2: Filter URLs


🔍 Classifying 643 headlines using LLM...


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",0
1,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,1
2,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",2
3,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,3
4,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,4
...,...,...,...,...,...,...
638,NewsAPI,AI runs on power. But power isn’t moving fast ...,https://www.wnd.com/2025/09/ai-runs-power-powe...,2025-09-21T19:19:36Z,NaN,638
639,NewsAPI,The Irish Times view on training and apprentic...,https://www.irishtimes.com/opinion/editorials/...,2025-09-21T19:31:00Z,NaN,639
640,NewsAPI,The Race to Reinvent U.S. Nuclear Power,https://oilprice.com/Alternative-Energy/Nuclea...,2025-09-21T19:00:00Z,NaN,640
641,NewsAPI,Reese Witherspoon Says Hollywood Will ‘Change ...,https://variety.com/2025/tv/news/reese-withers...,2025-09-21T19:15:52Z,NaN,641


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=465, user_len=954
16:22:07 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Initialized LLMagent:
system_prompt: You are a content-classification assistant that labels news headlines as AI-related or not.
You will receive a list of JSON object with fields "id" and "text_content"
Return **only** a JSON object that satisfies the provided schema.
For each headline provided, you MUST return one element with the same id, and a boolean value; do not skip any items.
Return elements in the same order they were provided.
No markdown, no markdown fences, no extra keys, no comments.
user_prompt: Classify every headline below.

AI-related if the title mentions (explicitly or implicitly):
- Core AI technologies: machine learning, neural / deep / transformer networks
- AI Applications: computer v

16:22:07 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | User message: Classify every headline below.

AI-related if the title mentions (explicitly or implicitly):
- Core AI technologies: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: ChatGPT, Gemini, Claude, Sora, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other news with a significant AI component

Non-AI examples: crypto, ordinary software, non-AI gadgets and medical device

...

16:22:21 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[AIClassification(id=425, input_str="Moody's raises Big Red flag over Oracle's AI datacenter buildout blueprint", output=True), AIClassification(id=426, input_str='FOMO? Brit banking biz rolls out AI tools, talks up security', output=True), AIClassification(id=427, input_str='Huawei used its own silicon to re-educate DeepSeek so its output won’t bother Beijing', output=True), AIClassification(id=428, input_str='Britain jumps into bed with Palantir in £1.5B defense pact', output=False), AIClassification(id=429, input_str='ChatGPT joins human league, now solves CAPTCHAs for the right prompt', output=True), AIClassification(id=430, input_str='Sorry, but DeepSeek didn’t really train its flagship model for $294,000', output=True), AIClassification(id=431, input_str="Microsoft insists Copilot+ PCs are 'empowering the future' – reality disagrees", output=True), AIClassification(id=432, input_str='Nic

16:22:23 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[AIClassification(id=175, input_str='Meta Connect 2025: Everything to expect', output=False), AIClassification(id=176, input_str='Microsoft announces a $30B investment in the UK over four years to support AI infrastructure and ongoing operations, including $15B to build a supercomputer (Tom Warren/The Verge)', output=True), AIClassification(id=177, input_str='Jack Altman raised a new $275M early-stage fund in a mere week', output=False), AIClassification(id=178, input_str='YouTube to use AI to help podcasters promote themselves with clips and Shorts', output=True), AIClassification(id=179, input_str='Anthropic report shows scale of AI threat to jobs', output=True), AIClassification(id=180, input_str='Humanoid robot startup Figure raises $1B+ at $39B valuation', output=True), AIClassification(id=181, input_str='Online services marketplace Fiverr lays off 30% of its workforce, or about 250 peopl

16:28:09 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[AIClassification(id=575, input_str='Trump says Rupert and Lachlan Murdoch likely part of US TikTok deal', output=False), AIClassification(id=576, input_str='Meta expos\x00e9 author faces bankruptcy after ban on criticising company', output=False), AIClassification(id=577, input_str='Chatbot site depicting child sexual abuse images raises fears over misuse of AI', output=True), AIClassification(id=578, input_str='Parents outraged as Meta uses photos of schoolgirls in ads targeting man', output=False), AIClassification(id=579, input_str='British AI startup beats humans in international forecasting competition', output=True), AIClassification(id=580, input_str='Inside the Jaguar Land Rover hack: stalled smart factories, outsourced cybersecurity and supply chain woes', output=False), AIClassification(id=581, input_str='Documents offer rare insight on Ice\x00e2\x0080\x0099s close relationship with

16:28:09 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Completed Step 2: Filtered to 343 AI-related articles


✅ Completed Step 2: Filtered to 343 AI-related headlines from 643 total
⏱️  Total execution time: 361.71s
📊 Final result:
✅ Step 2 completed successfully! Filtered 643 headlines to 343 AI-related articles.

📊 Results stored in persistent state. Current step: step_01_fetch_urls


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

16:29:46 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Starting Step 3: Download Articles


⏱️  Total execution time: 0.00s
📊 Final result:
Step 3 already completed! Filtered 304 articles, 304 identified as AI-related.


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Processing 304 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1204, user_len=43
16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Using model 'gpt-4.1-mini' for summarization
16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Initialized LLMagent:
system_prompt: You are an expert AI news analyst. Your task is to create concise, informative bullet-point summaries of AI and technology articles for a professional newsletter audience.

You will receive a list of JSON object with fields "id" and "title"
Return **only** a JSON object that satisfies the provided schema.
For each article provided, you MUST return one element with the same id, and the summary.
Return elements in the same order t

16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | User message: Summarize the article below:

[
  {
    "id":244,
    "text_content":"Page title: GitHub - QwenLM\/Qwen3-Omni: Qwen3-omni is a natively end-to-end, omni-modal LLM developed by the Qwen team at Alibaba Cloud, capable of understanding text, audio, images, and video, as well as generating speech in real time.\nSocial card title: GitHub - QwenLM\/Qwen3-Omni: Qwen3-omni is a natively end-to-end, omni-modal LLM developed by the Qwen team at Alibaba Cloud, capable of understanding text, audio, images, and video, as well as generating speech in real time.\nSocial card description: Qwen3-omni is a natively end-to-end, omni-modal LLM developed by the Qwen team at Alibaba Cloud, capable of understanding text, audio, images, and video, as well as generating speech in real time. ...\n\ud83d\udc9c Qwen Chat | \ud83e\udd17 Hugging Face | \ud83e\udd16 ModelScope | \ud83d\udcd1 Blog | \ud83d\udcda Cookbooks | \ud83d\

16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | User message: Summarize the article below:

[
  {
    "id":179,
    "text_content":"Page title: Anthropic report shows scale of AI threat to jobs | Semafor\nSocial card title: Anthropic report shows scale of AI threat to jobs\nSocial card description: Three-quarters of companies working with Claude, Anthropic\u2019s chatbot, use it for \u2018full task delegation.\u2019\nBusinesses use AI for automation more than collaboration, according to a report by Anthropic, intensifying fears the technology poses a greater threat to human jobs than previously thought.\nThree-quarters of companies working with Claude, Anthropic\u2019s chatbot, use it for \u201cfull task delegation\u201d \u2014 doing entire pieces of work, such as writing reports or coding software \u2014 rather than asking for information, or cooperating with it, the AI firm said.\nAnthropic\u2019s CEO has long predicted that AI will wipe out jobs and said six

16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | User message: Summarize the article below:

[
  {
    "id":260,
    "text_content":"Page title: Rohan Yadav\nSocial card title: Unweaving Warp Specialization\nSocial card description: \nRecently, I have been thinking deeply about warp specialization in the context of high performance kernels for modern Tensor Core GPUs like NVIDIA\u2019s H100 and B200. My understanding of what warp specialization achieves has deepened and led me to the interesting question of: do we actually need warp specialization (and the complexity that it entails)? My conclusion is that the answer is indeed yes, but it might not be as mandatory as it seems. In this post, I\u2019ll discuss when warp specialization is actually necessary, and describe the underlying trade-off space that I believe warp specialization resides within. While I will give some context on GPUs as necessary for discussing the topics at hand, this won\u2019t be a tutoria

16:28:55 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | User message: Summarize the article below:

[
  {
    "id":291,
    "text_content":"Page title: New frontiers in Human AI Interface | HackerNoon\nSocial card title: New frontiers in Human AI Interface | HackerNoon\nSocial card description: Recent tech advances are breaking free from 20 years of 5-inch screen limits, unlocking full human senses in computing through AI interfaces and wearables.\nIntroduction\nFor two decades, our interaction with the digital world has been confined to a mere 5-inch screen and a single fingertip. But what if we could break free from these limitations and unlock the full spectrum of our innate human senses in our everyday computing?\nThe past few years have witnessed a dramatic acceleration in the development of human-AI interface technologies, pushing the boundaries of how we interact with artificial intelligence. From immersive display technologies to intuitive wearable devices and 

...

16:29:04 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=199, summary="- Three Cascade High School students in eastern Iowa have been charged with creating fake nude images of other students using AI, with a fourth student facing expected charges; the images were created by attaching victims' headshots to nude body images.\n- The Dubuque County Attorney's Office and local law enforcement are involved, noting that the photos were circulated among students but likely did not spread beyond those circles; efforts are ongoing to remove the images from the internet.\n- School officials have banned charged students from in-person attendance at Cascade Junior/Senior High School, while authorities plan to educate students and parents about online dangers and monitoring children's digital activity amid this case.")]
16:29:04 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=483, summary=

16:29:04 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=598, summary="- Tanush Sharanarthi, a software engineer at IBM, initially failed to secure an H-1B visa three times due to the lottery system and instead successfully obtained an O-1 visa, known as the 'Einstein visa,' for individuals with extraordinary abilities in AI.\n- His six years of work, including research, open-source contributions, judging hackathons, and reviewing papers, aligned with the O-1 visa criteria, highlighting the link between sustained professional contributions and visa eligibility.\n- Sharanarthi advises aspiring O-1 applicants to focus on genuine contributions and passion in their field, underscoring networking's role in awareness and the importance of long-term dedication to meet the visa requirements.")]
16:29:04 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=462, summary="- The Really Simple

16:29:05 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=394, summary='- Song-Chun Zhu, a leading AI scientist who spent nearly three decades in the US, returned to China in 2020 to lead research at top Beijing universities and a state-sponsored AI institute, reflecting shifting global talent flows amid US-China tensions and changing scientific environments.\n- Zhu\'s AI philosophy challenges the dominant neural network paradigm prevalent in US tech companies, advocating for a "small data, big task" approach emphasizing reasoning and cognitive architectures over data-intensive neural networks like those behind ChatGPT.\n- Zhu\'s move symbolizes broader geopolitical and academic shifts: US research funding constraints, visa suspensions targeting Chinese scholars, and China\'s strategic investment to integrate AI deeply into its economy, while Zhu pursues AI advancements aligned with ethical priorities and China\'s centralized plann

16:29:05 | NewsletterAgent.test_newsletter_20250922162121167438 | INFO | Result: results_list=[ArticleSummary(id=321, summary='- AI adoption is rapidly accelerating, with 40% of U.S. employees using AI at work, surpassing adoption rates of prior technologies like the internet and personal computers.\n- AI usage is unevenly distributed globally and regionally, concentrated in high-income, technologically advanced areas such as Israel, Singapore, and parts of the U.S., while emerging economies lag significantly, raising concerns about exacerbated economic inequality.\n- Enterprise AI adoption focuses heavily on automation despite early-stage penetration (9.7% of U.S. firms), with key challenges including data modernization and contextual information access; strategic efforts are urged to promote inclusive access, workforce adaptability, and policy engagement to mitigate inequality risks and leverage AI’s economic potential.')]
16:29:05 | NewsletterAgent.test_newsletter_202509221621211674

✅ Completed Step 4: Generated AI summaries for 304/304 articles
⏱️  Total execution time: 44.97s
📊 Final result:
✅ Step 4 completed successfully! Generated AI-powered summaries for 304/304 articles.
💾 Summaries stored in headline DataFrame.


In [28]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


⏱️  Total execution time: 0.02s
📊 Final result:
❌ Step 5 failed: 'isAI'


In [24]:
agent.state.clear_errors()

In [27]:
agent.state.complete_step("step_01_fetch_urls")
agent.state.complete_step("step_02_filter_urls")
agent.state.complete_step("step_03_download_articles")
agent.state.complete_step("step_04_extract_summaries")


In [29]:
headline_df = pd.DataFrame(state.headline_data)
articles_with_summaries = headline_df.loc[
    (headline_df['isAI'] == True) &
    headline_df['summary'].notna() &
    (headline_df['summary'] != '')]


In [22]:
countdf = headline_df \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
4,Feedly AI,60
12,The Register,39
6,HackerNoon,37
3,FT,27
0,Ars Technica,20
1,Bloomberg,19
16,Washington Post,17
13,The Verge,15
14,VentureBeat,13
10,Techmeme,11


In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
state.save_checkpoint(session_id, "cluster_by_topic")


In [ ]:
agent.session.session_id

In [ ]:
from news_agent import ClusterByTopicTool
z = ClusterByTopicTool()
await z._canonical_topic_extraction(articles_with_summaries)


In [ ]:
    async def _free_form_extraction(articles_with_summaries: pd.DataFrame) -> pd.DataFrame:
            """Extract topics from article summaries using AI"""
            # Step 1: Extract topics from summaries using AI
            # Get prompt and model from Langfuse
            system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/extract_topics")

            # Create LLMagent for topic extraction
            topic_agent = LLMagent(
                system_prompt=system_prompt,
                user_prompt=user_prompt,
                output_type=TopicExtractionList,
                model=model,
                verbose=True,
                logger=logger
            )

            # Extract topics using filter_dataframe
            articles_with_summaries['extracted_topics'] = await topic_agent.filter_dataframe(
                articles_with_summaries[['id', 'summary']],
                value_field='topics_list',
                item_list_field='results_list',
                item_id_field='id',
                chunk_size=10
            )
            # Handle NaN values in extracted_topics
            articles_with_summaries['extracted_topics'] = articles_with_summaries['extracted_topics'].fillna('').apply(
                lambda x: list() if not isinstance(x, list) else x
            )


            return articles_with_summaries

In [ ]:

class TopicExtraction(BaseModel):
    """Topic extraction result for a single article"""
    id: int = Field(description="The article id")
    topics_list: List[str] = Field(description="List of relevant topics discussed in the article")

class TopicExtractionList(BaseModel):
    """List of TopicExtraction for batch processing"""
    results_list: list[TopicExtraction] = Field(description="List of topic extraction results")

articles_with_summaries = await _free_form_extraction(articles_with_summaries)


In [ ]:
    async def _classify_canonical_topic(headline_df: pd.DataFrame, topic: str) -> List[bool]:
        """
        Classify all summaries against a single canonical topic

        Args:
            headline_df: DataFrame with articles containing summaries
            topic: Single canonical topic to classify against

        Returns:
            List of boolean values indicating relevance to the topic
        """
        # Get prompt and model from Langfuse
        langfuse_client = LangfuseClient()
        system_prompt, user_prompt, model = langfuse_client.get_prompt("newsagent/canonical_topic")

        # Create LLMagent for canonical topic classification
        canonical_agent = LLMagent(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            output_type=CanonicalTopicClassificationList,
            model=model,
            verbose=False, #  too much output , always false
            logger=logger
        )

        # Use filter_dataframe to classify against the canonical topic
        relevance_series = await canonical_agent.filter_dataframe(
            headline_df[['id', 'summary']].copy(),
            value_field='relevant',
            item_list_field='results_list',
            item_id_field='id',
            input_vars={'topic': topic},
            chunk_size=10
        )

        return topic, relevance_series.tolist()

In [ ]:
from config import CANONICAL_TOPICS

# Canonical topic classification models
class CanonicalTopicClassification(BaseModel):
    """Single article classification result for a canonical topic"""
    id: int = Field(description="The article id")
    relevant: bool = Field(description="Whether the summary is relevant to the canonical topic")

class CanonicalTopicClassificationList(BaseModel):
    """List of classification results for batch processing"""
    results_list: list[CanonicalTopicClassification] = Field(description="List of classification results")


In [ ]:
await _classify_canonical_topic(articles_with_summaries, "Anthropic")


In [ ]:

        """Extract canonical topics from article summaries using AI"""
        # Classify against canonical topics
        # Create all canonical topic classification tasks
        canonical_tasks = [
            _classify_canonical_topic(articles_with_summaries, topic)
            for topic in CANONICAL_TOPICS
        ]

        # Run all canonical topic classifications concurrently
        canonical_results = await asyncio.gather(*canonical_tasks, return_exceptions=True)
        print(canonical_results)
        # Initialize canonical_topics as list of empty lists
        canonical_topics_lists = [list() for _ in range(len(articles_with_summaries))]

        # Process canonical results
        for result in canonical_results:
            if isinstance(result, Exception):
                if self.logger:
                    self.logger.warning(f"Topic classification failed: {result}")
                continue

            topic, relevance_list = result
            if isinstance(relevance_list, list):
                for idx, is_relevant in enumerate(relevance_list):
                    if is_relevant:
                        canonical_topics_lists[idx].append(topic)
        
        # Assign to canonical_topics column
        articles_with_summaries['canonical_topics'] = canonical_topics_lists



In [ ]:
articles_with_summaries


In [ ]:
articles_with_summaries = await _canonical_topic_extraction(articles_with_summaries)


In [ ]:
state.clear_errors()


In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 5, cluster by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
headline_df = pd.DataFrame(state.headline_data)
headline_df

In [ ]:
            headline_df = pd.DataFrame(state.headline_data)
            articles_with_summaries = headline_df.loc[
                (headline_df['isAI'] == True) &
                headline_df['summary'].notna() &
                (headline_df['summary'] != '')]

        

In [ ]:
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/canonical_topic")
print(system_prompt)
print(user_prompt)
model


In [ ]:
class CanonicalTopicClassification(BaseModel):
    """Single article classification result for a canonical topic"""
    id: int = Field(description="The article id")
    relevant: bool = Field(description="Whether the summary is relevant to the canonical topic")

class CanonicalTopicClassificationList(BaseModel):
    """List of classification results for batch processing"""
    results_list: list[CanonicalTopicClassification] = Field(description="List of classification results")



In [ ]:
        canonical_agent = LLMagent(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            output_type=CanonicalTopicClassificationList,
            model=model,
            verbose=True,
            logger=logger
        )

    

In [ ]:
async def classify_canonical_topic(headline_df: pd.DataFrame, topic: str) -> List[bool]:
        """
        Classify all summaries against a single canonical topic

        Args:
            headline_df: DataFrame with articles containing summaries
            topic: Single canonical topic to classify against

        Returns:
            List of boolean values indicating relevance to the topic
        """
        # Get prompt and model from Langfuse
        langfuse_client = LangfuseClient()
        system_prompt, user_prompt, model = langfuse_client.get_prompt("newsagent/canonical_topic")

        # Create LLMagent for canonical topic classification
        canonical_agent = LLMagent(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            output_type=CanonicalTopicClassificationList,
            model=model,
            verbose=True,
            logger=logger
        )

        # Use filter_dataframe to classify against the canonical topic
        relevance_series = await canonical_agent.filter_dataframe(
            headline_df[['id', 'summary']].copy(),
            value_field='relevant',
            item_list_field='results_list',
            item_id_field='id',
            input_vars={'topic': topic},
            chunk_size=10
        )

        return topic, relevance_series.tolist()

In [ ]:
await classify_canonical_topic(headline_df.copy(), "Anthropic")


In [ ]:
from config import CANONICAL_TOPICS


In [ ]:
async def canonical_topic_extraction(articles_with_summaries: pd.DataFrame) -> pd.DataFrame:
    """Extract canonical topics from article summaries using AI"""
    # Classify against canonical topics
    logger.info(f"Starting canonical topic classification for {len(CANONICAL_TOPICS)} topics")

    # Create all canonical topic classification tasks
    canonical_tasks = [
        classify_canonical_topic(articles_with_summaries, topic)
        for topic in CANONICAL_TOPICS
    ]

    # Run all canonical topic classifications concurrently
    canonical_results = await asyncio.gather(*canonical_tasks, return_exceptions=True)
    # Initialize canonical_topics as list of empty lists
    canonical_topics_lists = [list() for _ in range(len(articles_with_summaries))]

    # Process canonical results
    for topic, relevance_list in canonical_results:
        if isinstance(relevance_list, list):
            for idx, is_relevant in enumerate(relevance_list):
                if is_relevant:
                    canonical_topics_lists[idx].append(topic)

    # Assign to canonical_topics column
    articles_with_summaries['canonical_topics'] = canonical_topics_lists

    total_canonical_matches = sum(len(topics) for topics in articles_with_summaries['canonical_topics'])
    logger.info(f"Canonical topic classification complete: {total_canonical_matches} total topic matches")

    return articles_with_summaries



In [ ]:
await canonical_topic_extraction(headline_df.copy())


In [ ]:
print("manza")


In [ ]:
# Topic extraction models
class TopicExtraction(BaseModel):
    """Topic extraction result for a single article"""
    id: int = Field(description="The article id")
    topics_list: List[str] = Field(description="List of up to 5 distinct, broad topics")

class TopicExtractionList(BaseModel):
    """List of TopicExtraction for batch processing"""
    results_list: list[TopicExtraction] = Field(description="List of topic extraction results")


In [ ]:
async def _free_form_extraction(articles_with_summaries: pd.DataFrame) -> pd.DataFrame:
        """Extract topics from article summaries using AI"""
        # Step 1: Extract topics from summaries using AI
        logger.info(f"Starting topic extraction for clustering")

        # Get prompt and model from Langfuse
        system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/extract_topics")

        logger.info(f"Using model '{model}' for topic extraction")
        logger.info(f"Processing {len(articles_with_summaries)} articles for topic extraction")

        # Create LLMagent for topic extraction
        topic_agent = LLMagent(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            output_type=TopicExtractionList,
            model=model,
            verbose=True,
            logger=logger
        )

        # Extract topics using filter_dataframe
        articles_with_summaries['extracted_topics'] = await topic_agent.filter_dataframe(
            articles_with_summaries[['id', 'summary']],
            value_field='topics_list',
            item_list_field='results_list',
            item_id_field='id'
        )
        # Handle NaN values in extracted_topics
        articles_with_summaries['extracted_topics'] = articles_with_summaries['extracted_topics'].fillna('').apply(
            lambda x: listt() if not isinstance(x, list) else x
        )

        topics_extracted = articles_with_summaries['extracted_topics'].apply(lambda x: len(x) if isinstance(x, list) else 0).sum()
        logger.info(f"Successfully extracted {topics_extracted} total topics across articles")

        return articles_with_summaries


In [ ]:
topics_extracted = headline_df['extracted_topics'].apply(lambda x: len(x) if isinstance(x, list) else 0).sum()
topics_extracted


In [ ]:
await _free_form_extraction(headline_df)


In [ ]:
# Create LLMagent for topic extraction
topic_agent = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=TopicExtractionList,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
# Extract topics using filter_dataframe
headline_df['extracted_topics'] = await topic_agent.filter_dataframe(
    headline_df[['id', 'summary']],
    value_field='topics_list',
    item_list_field='results_list',
    item_id_field='id'
)


In [ ]:
headline_df


- free-form topic extraction
- find frequent topics and add to canonical and output them
- does match canonical topics with 
- filter topics select up to 7
-                                       TOPIC_FILTER_SYSTEM_PROMPT, TOPIC_FILTER_USER_PROMPT,
- update summaries, add topics
- cluster  articles
- fetch embeddings for summaries
- do dimensionality reduction
- cluster with hdbscan
- show metrics
- name the clusters with topic_writer
- store the cluster names 

output is , df has topic list and topic_str, summary updated, df has cluster , state clusters updated

In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
state = await agent.get_state_direct()


In [ ]:
agent = NewsletterAgent(session_id="test_newsletter_20250918142308630453", verbose=True)
status_result = await agent.run_step("check workflow status")


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
print(status_result)


In [ ]:
# todos
# be less verbose
# show current headlines 
# try to call detailed state inspection tool from a prompt , showing count by source 
# get prompts from langfuse

In [ ]:
# class to store agent state from step to step
from newsletter_state import NewsletterAgentState


In [ ]:
import sys 

# if 'fetch' in sys.modules:
#     del sys.modules['fetch']
#     # Delete the reference
#     del Fetcher
from fetch import Fetcher

# should probably do this in the initialization based on parameters --nofetch
destination = "download/sources/"
for file in os.listdir(destination):
    file_path = os.path.join(destination, file)
    if os.path.isfile(file_path):
        os.remove(file_path)
        logger.info(f"Removed existing file: {file_path}")

In [ ]:
async with Fetcher() as f:
     z = await f.fetch_all()
z 


In [ ]:
len(z)

In [ ]:
for src in z:
    print(src['source'])
    print(src['status'])
    print(src['metadata'])
    print(len(src['results']))
          

In [ ]:
z[0]

In [ ]:
sources_results = z
successful_sources = []
failed_sources = []
all_articles = []

for result in sources_results:
    if result['status'] == 'success' and result['results']:
        # Add source info to each article
        successful_sources.append(result['source'])
        all_articles.extend(result['results'])
    else:
        failed_sources.append(result['source'])
        
headline_data = all_articles

In [ ]:
# TODO
len(all_articles)
headline_df = pd.DataFrame(all_articles)
display(headline_df[["source", "url"]].groupby("source") \
    .count() \
    .reset_index() \
    .rename({'url': 'count'}))


In [ ]:
f.sources.get('Ars Technica')


In [ ]:
from news_agent import NewsletterAgent


In [ ]:
news_agent = NewsletterAgent(session_id=f"newsletter_{random.randint(10000000, 99999999)}", verbose=True)


In [ ]:
user_prompt = "Can you show me the current status of the newsletter workflow"

start_time = time.time()
result = await news_agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



In [ ]:
result


In [ ]:
# Create mock context
class MockContext:
    def __init__(self):
        self.context = news_agent.default_state

ctx = MockContext()
current_state = ctx.context  # From your previous run, or reload it
df = current_state.headline_df
df



In [ ]:
try:
    current_state = news_agent.session.get_state()
except:
    current_state = news_agent.default_state

print(current_state)
print()

print(f"Current Step: {current_state.current_step}/9")
print(f"Workflow Complete: {current_state.workflow_complete}")
print(f"Progress: {(current_state.current_step/9)*100:.1f}%")
print(f"Total articles: {len(current_state.headline_data)}")

if current_state.headline_data:
    ai_related = sum(1 for a in current_state.headline_data if a.get('ai_related') is True)
    print(f"AI-related articles: {ai_related}")
    print(f"Summaries: {len(current_state.article_summaries)}")
    print(f"Clusters: {len(current_state.topic_clusters)}")
    print(f"Sections: {len(current_state.newsletter_sections)}")

In [ ]:
# review slides

# review workflow status, move to a moadule
# all prints should be logs
# section writing and composition will have the critic /optimizer loop
# add batch with async


In [ ]:
def create_news_dataframe():
    """
    Creates an empty DataFrame to support headline/article analysis
    - URLs, source tracking and metadata
    - Topic classification and clustering
    - Content quality ratings and rankings

    Returns:
        pd.DataFrame: Empty DataFrame with predefined column structure
    """

    # column structure
    column_dict = {
        # Core identifiers and source info
        'article_id': 'object',              # Unique identifier for each article
        'source':     'object',              # Source category
        'headline_title': 'object',          # Article headline/title
        'original_url': 'object',            # Initial URL before redirects
        'final_url': 'object',               # URL after following redirects
        'domain_name': 'category',           # Website domain
        'site_name': 'category',             # Human-readable site name
        'site_reputation_score': 'float32',  # Reputation/trustworthiness score for the site
        'keep_flag': 'boolean',

        # File paths and storage
        'html_file_path': 'object',          # Path to stored HTML content
        'text_file_path': 'object',          # Path to extracted text content

        # Time information
        'last_updated_timestamp': 'datetime64[ns]',  # When article was last updated
        'article_age_days': 'int32',         # Age of article in days
        'recency_score': 'float32',          # Calculated recency score (higher = more recent)

        # Content analysis
        'content_summary': 'object',         # Generated summary of article content
        'bullet_points': 'object',           # Key points extracted as bullets
        'article_length_chars': 'int32',     # Character count of article content

        # Rating flags (LLM-generated probabilities)
        'is_high_quality': 'float32',        # LLM probability for low-quality content
        'is_off_topic': 'float32',           # LLM probability for off-topic content
        'is_low_importance': 'float32',      # 1-LLM probability for high-importance content

        # Other ratings
        'bradley_terry_score': 'float32',    # Bradley-Terry rating from pairwise article comparisons
        'bradley_terry_rank': 'int32',       # Ordinal rank based on Bradley-Terry scores (1 = highest rated)
        'adjusted_length_score': 'float32',  # Length-adjusted quality score
        'final_composite_rating': 'float32', # Final weighted rating combining multiple factors

        # Topic classification
        'topic_string': 'object',            # Topic labels as comma-separated string
        'topic_list': 'object',              # Topic labels as list/array structure (same topics, different format)

        # Organization and clustering (HDBSCAN-based)
        'display_order': 'int32',            # Order for display/presentation
        'cluster_id': 'int32',               # HDBSCAN cluster identifier (-1 = noise/outlier)
        'cluster_label': 'category'          # Human-readable cluster name/description
    }

    # Create empty DataFrame from column dictionary
    df = pd.DataFrame(columns=list(column_dict.keys())).astype(column_dict)

    return df



In [ ]:
@dataclass
class NewsletterState:
    """
    Maintains session state for the OpenAI Agents SDK workflow.

    Attributes:
        headline_df: DataFrame containing headline data for processing
        sources_file: Path to YAML file containing source configurations
        sources: Dictionary of source configurations loaded from YAML
        cluster_topics: List of clean topic names for headline categorization
        max_edits: Maximum number of critic optimizer editing iterations allowed
        edit_complete: Boolean flag indicating if editing process is finished
        n_browsers: Number of concurrent Playwright browser instances for downloads
    """

    status: WorkflowStatus = WorkflowStatus()
    headline_df: pd.DataFrame = field(default_factory=create_news_dataframe)
    sources_file: str = field(default="sources.yaml")
    sources: Dict[str, Any] = field(default_factory=dict)
    cluster_topics: List[str] = field(default_factory=list)
    max_edits: int = field(default=3)
    edit_complete: bool = field(default=False)
    n_browsers: int = field(default=8)
    verbose: bool = field(default=True)


    def __post_init__(self):
        """
        Post-initialization validation and setup.

        Validates that the configuration makes sense and performs
        any necessary initialization steps.
        """
        # Validate max_edits is reasonable
        if self.max_edits < 1 or self.max_edits > 10:
            raise ValueError(f"max_edits should be between 1-10, got {self.max_edits}")

        # Validate n_browsers is reasonable
        if self.n_browsers < 1 or self.n_browsers > 32:
            raise ValueError(f"n_browsers should be between 1-32, got {self.n_browsers}")

        # Validate sources_file exists and load sources from file automatically
        try:
            sources_path = Path(self.sources_file)
            with open(sources_path, 'r', encoding='utf-8') as file:
                self.sources = yaml.safe_load(file) or {}
            if self.verbose:
                print(f"Loaded {len(self.sources)} sources from {self.sources_file}")
        except FileNotFoundError:
            raise FileNotFoundError(f"Sources file not found: {self.sources_file}")
        except yaml.YAMLError as e:
            raise ValueError(f"Error parsing YAML file {self.sources_file}: {e}")


In [ ]:
state = NewsletterState()
state


In [ ]:
from agents import Agent, Runner, SQLiteSession, function_tool, RunContextWrapper


In [ ]:
class NewsletterAgent(Agent[NewsletterState]):
    """AI newsletter writing agent with structured workflow"""

    def __init__(self, session_id: str = "newsletter_agent"):
        self.session = SQLiteSession(session_id, "newsletter.db")
        self.state = NewsletterState()

        super().__init__(
            name="AINewsletterAgent",
            instructions="""
            You are an AI newsletter writing agent. Your role is to:
            1. Scrape headlines and URLs from various sources
            2. Filter the headlines to ones that are about AI
            3. Fetch the URLs and save them as plain text
            4. Summarize each article to 3 bullet points containing the key facts
            5. Extract topics from each article and cluster articles by topic
            6. Rate each article according to the provided rubric
            7. Identify 6-15 thematic sections + "Other News", assign articles to sections and deduplicate
            8. Write each section
            9. Combine sections and polish

            Use the tools available to accomplish these tasks in order.
            Always maintain context about workflow progress and data.
            Guide users through the workflow steps systematically.
            """,
            tools=[
                self.step1_scrape_headlines,
                self.step2_filter_ai_headlines,
                self.step3_fetch_article_texts,
                self.step4_summarize_articles,
                self.step5_extract_and_cluster_topics,
                self.step6_rate_articles,
                self.step7_organize_sections,
                self.step8_write_sections,
                self.step9_finalize_newsletter,
                self.get_workflow_status,
                self.run_complete_workflow,
                self.reset_workflow
            ]
        )

    @function_tool
    async def step1_scrape_headlines(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        sources: List[str] = None,
        max_articles_per_source: int = 50
    ) -> str:
        """Step 1: Scrape headlines and URLs from various sources"""
        if sources is None:
            sources = ["techcrunch", "arstechnica", "theverge", "wired", "venturebeat"]

        scraped_data = []

        # Mock scraping implementation (replace with real RSS/API scraping)
        for source in sources:
            for i in range(max_articles_per_source):
                article = {
                    'title': f"{source} AI Article {i+1}: Latest developments in machine learning",
                    'url': f"https://{source}.com/ai-article-{i+1}",
                    'source': source,
                    'published_at': (datetime.now() - timedelta(hours=i)).isoformat(),
                    'description': f"AI-related content from {source}"
                }
                scraped_data.append(article)

        wrapper.context.raw_headlines = scraped_data
        wrapper.context.scraped_urls = [article['url'] for article in scraped_data]
        wrapper.context.current_step = 1

        return f"✅ Step 1 Complete: Scraped {len(scraped_data)} headlines from {len(sources)} sources"


    @function_tool
    async def step2_filter_ai_content(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        ai_keywords: List[str] = None
    ) -> str:
        """Step 2: Filter headlines to AI-related content only"""
        if not wrapper.context.raw_headlines:
            return "❌ No headlines to filter. Run step 1 first."

        if ai_keywords is None:
            ai_keywords = [
                'ai', 'artificial intelligence', 'machine learning', 'deep learning',
                'neural network', 'llm', 'gpt', 'transformer', 'chatbot', 'automation',
                'computer vision', 'nlp', 'natural language', 'algorithm', 'model'
            ]

        ai_articles = []
        for article in wrapper.context.raw_headlines:
            title_lower = article['title'].lower()
            desc_lower = article['description'].lower()

            # Check if any AI keywords are present
            if any(keyword in title_lower or keyword in desc_lower for keyword in ai_keywords):
                ai_articles.append(article)

        wrapper.context.ai_headlines = pd.DataFrame(ai_articles)
        wrapper.context.current_step = 2

        return f"✅ Step 2 Complete: Filtered to {len(ai_articles)} AI-related headlines from {len(wrapper.context.raw_headlines)} total"

    @function_tool
    async def step3_fetch_article_texts(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 3: Fetch full article texts from URLs"""
        if wrapper.context.ai_headlines.empty:
            return "❌ No AI headlines to fetch. Complete steps 1-2 first."

        # Mock article fetching (replace with actual web scraping)
        article_texts = {}

        for _, row in wrapper.context.ai_headlines.iterrows():
            url = row['url']
            # Mock article content
            article_texts[url] = f"""
            {row['title']}

            This is a mock article about AI developments. In a real implementation,
            you would use libraries like requests + BeautifulSoup or newspaper3k
            to extract the full article text from the URL.

            Key points about this AI story:
            - Advancement in machine learning techniques
            - Impact on industry applications
            - Future implications for AI development

            This content would be much longer in practice, containing the full
            article text that needs to be summarized and analyzed.
            """

        wrapper.context.article_texts = article_texts
        wrapper.context.current_step = 3

        return f"✅ Step 3 Complete: Fetched full text for {len(article_texts)} articles"

    @function_tool
    async def step4_summarize_articles(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 4: Summarize each article to 3 key bullet points"""
        if not wrapper.context.article_texts:
            return "❌ No article texts to summarize. Complete steps 1-3 first."

        summaries = {}

        for url, text in wrapper.context.article_texts.items():
            # Mock summarization (replace with actual LLM summarization)
            summaries[url] = [
                "• Key development in AI technology or research",
                "• Practical implications for businesses or developers",
                "• Future outlook or next steps in this area"
            ]

        wrapper.context.article_summaries = summaries
        wrapper.context.current_step = 4

        return f"✅ Step 4 Complete: Generated 3-point summaries for {len(summaries)} articles"

    @function_tool
    async def step5_extract_and_cluster_topics(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        max_clusters: int = 8
    ) -> str:
        """Step 5: Extract topics and cluster articles"""
        if not wrapper.context.article_texts:
            return "❌ No articles to analyze. Complete steps 1-4 first."

        # Extract topics from each article (mock implementation)
        article_topics = {}
        all_topics = []

        for url, text in wrapper.context.article_texts.items():
            # Mock topic extraction (replace with NLP)
            topics = ['machine learning', 'business applications', 'research', 'ethics']
            article_topics[url] = topics
            all_topics.extend(topics)

        # Cluster articles by common topics
        topic_counts = Counter(all_topics)
        main_topics = [topic for topic, count in topic_counts.most_common(max_clusters)]

        topic_clusters = {}
        for topic in main_topics:
            topic_clusters[topic] = [
                url for url, topics in article_topics.items()
                if topic in topics
            ]

        wrapper.context.article_topics = article_topics
        wrapper.context.topic_clusters = topic_clusters
        wrapper.context.current_step = 5

        return f"✅ Step 5 Complete: Extracted topics and created {len(topic_clusters)} clusters"

    @function_tool
    async def step6_rate_articles(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        custom_rubric: Dict[str, str] = None
    ) -> str:
        """Step 6: Rate articles according to rubric"""
        if not wrapper.context.article_texts:
            return "❌ No articles to rate. Complete previous steps first."

        if custom_rubric:
            wrapper.context.rating_rubric.update(custom_rubric)

        # Mock rating (replace with actual evaluation)
        ratings = {}
        for url in wrapper.context.article_texts.keys():
            # Mock scoring based on rubric criteria
            relevance_score = 0.8
            novelty_score = 0.7
            impact_score = 0.9
            credibility_score = 0.8

            overall_rating = (relevance_score + novelty_score + impact_score + credibility_score) / 4
            ratings[url] = overall_rating

        wrapper.context.article_ratings = ratings
        wrapper.context.current_step = 6

        avg_rating = sum(ratings.values()) / len(ratings)
        return f"✅ Step 6 Complete: Rated {len(ratings)} articles. Average rating: {avg_rating:.2f}"

    @function_tool
    async def step7_organize_sections(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        target_sections: int = 10
    ) -> str:
        """Step 7: Organize articles into thematic sections"""
        if not wrapper.context.topic_clusters:
            return "❌ No topic clusters available. Complete steps 1-6 first."

        # Create thematic sections based on clusters and ratings
        sections = {}

        # Main thematic sections from top clusters
        top_clusters = sorted(
            wrapper.context.topic_clusters.items(),
            key=lambda x: len(x[1]),  # Sort by cluster size
            reverse=True
        )[:target_sections-1]  # Reserve space for "Other News"

        for topic, urls in top_clusters:
            # Only include high-rated articles
            high_rated_urls = [
                url for url in urls
                if wrapper.context.article_ratings.get(url, 0) >= 0.6
            ]
            if high_rated_urls:
                section_name = topic.title().replace('_', ' ')
                sections[section_name] = high_rated_urls

        # "Other News" section for remaining articles
        assigned_urls = set()
        for urls in sections.values():
            assigned_urls.update(urls)

        other_urls = [
            url for url in wrapper.context.article_texts.keys()
            if url not in assigned_urls and wrapper.context.article_ratings.get(url, 0) >= 0.5
        ]

        if other_urls:
            sections["Other News"] = other_urls

        wrapper.context.thematic_sections = sections
        wrapper.context.section_names = list(sections.keys())
        wrapper.context.current_step = 7

        section_summary = "\n".join([
            f"• {name}: {len(urls)} articles"
            for name, urls in sections.items()
        ])

        return f"✅ Step 7 Complete: Organized into {len(sections)} sections:\n{section_summary}"

    @function_tool
    async def step8_write_sections(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 8: Write content for each thematic section"""
        if not wrapper.context.thematic_sections:
            return "❌ No sections to write. Complete steps 1-7 first."

        section_drafts = {}

        for section_name, urls in wrapper.context.thematic_sections.items():
            # Gather content for this section
            section_articles = []

            for url in urls:
                summary = wrapper.context.article_summaries.get(url, [])
                rating = wrapper.context.article_ratings.get(url, 0)

                # Get article title from DataFrame
                article_row = wrapper.context.ai_headlines[
                    wrapper.context.ai_headlines['url'] == url
                ]
                title = article_row['title'].iloc[0] if not article_row.empty else "Unknown Title"

                section_articles.append({
                    'title': title,
                    'url': url,
                    'summary': summary,
                    'rating': rating
                })

            # Write section content (mock implementation)
            section_content = f"## {section_name}\n\n"

            for article in sorted(section_articles, key=lambda x: x['rating'], reverse=True):
                section_content += f"**{article['title']}**\n"
                for bullet in article['summary']:
                    section_content += f"{bullet}\n"
                section_content += f"[Read more]({article['url']})\n\n"

            section_drafts[section_name] = section_content

        wrapper.context.section_drafts = section_drafts
        wrapper.context.current_step = 8

        return f"✅ Step 8 Complete: Wrote content for {len(section_drafts)} sections"

    @function_tool
    async def step9_finalize_newsletter(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        newsletter_title: str = "AI Weekly Newsletter"
    ) -> str:
        """Step 9: Combine sections and polish final newsletter"""
        if not wrapper.context.section_drafts:
            return "❌ No section drafts available. Complete steps 1-8 first."

        # Combine all sections
        newsletter_content = f"# {newsletter_title}\n"
        newsletter_content += f"*Generated on {datetime.now().strftime('%B %d, %Y')}*\n\n"

        # Add introduction
        total_articles = len(wrapper.context.article_texts)
        newsletter_content += f"This week's AI newsletter covers {total_articles} key developments across {len(wrapper.context.section_drafts)} areas of AI.\n\n"

        # Add each section
        for section_name in wrapper.context.section_names:
            if section_name in wrapper.context.section_drafts:
                newsletter_content += wrapper.context.section_drafts[section_name]
                newsletter_content += "\n---\n\n"

        # Add footer
        newsletter_content += "*Thank you for reading! This newsletter was generated using AI curation and analysis.*"

        wrapper.context.final_newsletter = newsletter_content
        wrapper.context.workflow_complete = True
        wrapper.context.current_step = 9

        return f"✅ Step 9 Complete: Finalized newsletter with {len(wrapper.context.section_drafts)} sections"

    @function_tool
    async def get_workflow_status(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Get detailed workflow progress status"""
        state = wrapper.context

        status = {
            'current_step': state.current_step,
            'steps_completed': [
                f"1. Scraping: {len(state.raw_headlines)} headlines" if state.raw_headlines else "1. Scraping: Pending",
                f"2. AI Filtering: {len(state.ai_headlines)} AI articles" if not state.ai_headlines.empty else "2. AI Filtering: Pending",
                f"3. Text Fetching: {len(state.article_texts)} articles" if state.article_texts else "3. Text Fetching: Pending",
                f"4. Summarization: {len(state.article_summaries)} summaries" if state.article_summaries else "4. Summarization: Pending",
                f"5. Topic Clustering: {len(state.topic_clusters)} clusters" if state.topic_clusters else "5. Topic Clustering: Pending",
                f"6. Article Rating: {len(state.article_ratings)} rated" if state.article_ratings else "6. Article Rating: Pending",
                f"7. Section Organization: {len(state.thematic_sections)} sections" if state.thematic_sections else "7. Section Organization: Pending",
                f"8. Section Writing: {len(state.section_drafts)} drafts" if state.section_drafts else "8. Section Writing: Pending",
                f"9. Newsletter Finalization: {'Complete' if state.final_newsletter else 'Pending'}"
            ],
            'workflow_complete': state.workflow_complete
        }

        return f"Newsletter Workflow Status:\n\n" + "\n".join(status['steps_completed'])

    @function_tool
    async def run_complete_workflow(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        sources: List[str] = None,
        ai_keywords: List[str] = None
    ) -> str:
        """Run the complete 9-step workflow automatically"""
        results = []

        # Execute each step in sequence
        result1 = await self.step1_scrape_headlines(wrapper, sources)
        results.append(result1)

        result2 = await self.step2_filter_ai_content(wrapper, ai_keywords)
        results.append(result2)

        result3 = await self.step3_fetch_article_texts(wrapper)
        results.append(result3)

        result4 = await self.step4_summarize_articles(wrapper)
        results.append(result4)

        result5 = await self.step5_extract_and_cluster_topics(wrapper)
        results.append(result5)

        result6 = await self.step6_rate_articles(wrapper)
        results.append(result6)

        result7 = await self.step7_organize_sections(wrapper)
        results.append(result7)

        result8 = await self.step8_write_sections(wrapper)
        results.append(result8)

        result9 = await self.step9_finalize_newsletter(wrapper)
        results.append(result9)

        newsletter_length = len(wrapper.context.final_newsletter)

        return "\n".join(results) + f"\n\n🎉 Complete workflow finished! Newsletter ready ({newsletter_length} characters)"

    @function_tool
    async def reset_workflow(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Reset workflow to start fresh"""
        wrapper.context.__dict__.update(NewsletterState().__dict__)
        return "🔄 Workflow reset. Ready to start step 1."

    @function_tool
    async def get_newsletter_preview(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        max_chars: int = 500
    ) -> str:
        """Get a preview of the current newsletter"""
        if not wrapper.context.final_newsletter:
            return "Newsletter not ready yet. Complete the full workflow first."

        preview = wrapper.context.final_newsletter[:max_chars]
        if len(wrapper.context.final_newsletter) > max_chars:
            preview += "..."

        return f"Newsletter Preview:\n\n{preview}"

    async def run_step(self, user_input: str) -> str:
        """Run a workflow step with persistent state"""
        result = await Runner.run(
            self,
            user_input,
            session=self.session,
            context=self.state
        )
        return result.final_output

    def save_newsletter(self, filepath: str = None):
        """Save the final newsletter to file"""
        if not self.state.final_newsletter:
            print("No newsletter to save. Complete workflow first.")
            return

        if filepath is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"ai_newsletter_{timestamp}.md"

        with open(filepath, 'w') as f:
            f.write(self.state.final_newsletter)

        print(f"Newsletter saved to {filepath}")




In [ ]:
import openai

client = openai.OpenAI(
  base_url="http://localhost:8787/v1",
  api_key=os.getenv("OPENAI_API_KEY"),
  default_headers={"x-portkey-provider": "openai"}
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role": "user", "content": "Hello"}]
)
print(response.choices[0].message.content)

In [ ]:
from portkey_ai import Portkey

client = Portkey(
    provider="openai",
    Authorization=os.getenv("OPENAI_API_KEY")
)

# Example: Send a chat completion request
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello, how are you?"}],
    model="gpt-4o"
)

print(response.choices[0].message.content)

In [ ]:
type(prompt_template)

In [ ]:
class AgentState(TypedDict):
    """
    State of the LangGraph agent.
    Each node in the graph is a function that takes the current state and returns the updated state.
    """

    # the current working set of headlines (pandas dataframe not supported)
    AIdf: list[dict]
    # ignore stories before this date for deduplication (force reprocess since)
    model_low: str     # cheap fast model like gpt-4o-mini or flash
    model_medium: str  # medium model like gpt-4o or gemini-1.5-pro
    model_high: str    # slow expensive thinking model like o3-mini
    sources: dict  # sources to scrap
    sources_reverse: dict[str, str]  # map file names to sources

state = AgentState()


In [ ]:
SOURCES_FILE = "sources.yaml"

def initialize(state, sources_file=SOURCES_FILE) -> Dict[str, Any]:
    """Read and parse the sources.yaml file."""
    try:
        with open(sources_file, 'r', encoding='utf-8') as file:
            state["sources"] =  yaml.safe_load(file)
        state["sources_reverse"] = {v["title"]+".html":k for k,v in state["sources"].items()}
    except FileNotFoundError:
        raise FileNotFoundError(f"Sources file '{self.sources_file}' not found")
    except yaml.YAMLError as e:
        raise ValueError(f"Error parsing YAML file: {e}")

    return state


In [ ]:
state = initialize(state)
state
